In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
filepath = "D:\FPL-Lineup-Prediction\data\converted_data\gameweek_data.csv"

df = pd.read_csv(filepath)

<>:1: SyntaxWarning: invalid escape sequence '\F'
<>:1: SyntaxWarning: invalid escape sequence '\F'
C:\Users\ramgo\AppData\Local\Temp\ipykernel_45448\725384392.py:1: SyntaxWarning: invalid escape sequence '\F'
  filepath = "D:\FPL-Lineup-Prediction\data\converted_data\gameweek_data.csv"


In [3]:
df.head()

,Unnamed: 0,player_id,player_name,now_cost,position,team,gw1,gw2,gw3,gw4,gw5,gw6,gw7,gw8,gw9,gw10,gw11,gw12
0,0,1,Raya,59,GKP,Arsenal,10.0,6.0,2.0,6.0,2.0,2.0,6.0,6.0,6.0,6.0,1.0,2
1,1,2,Arrizabalaga,42,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2,3,Hein,40,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,3,4,Setford,40,GKP,Arsenal,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,5,Gabriel,64,DEF,Arsenal,6.0,6.0,2.0,9.0,2.0,13.0,9.0,12.0,9.0,12.0,1.0,0
